# Initialize Models
This notebook will walk you through building and saving the most basic 
models we used for analyzing our text data.

We first import the libraries and utility files we are going to be using.

In [1]:
# Import useful mathematical libraries
import numpy as np
import pandas as pd

# Import useful Machine learning libraries
import gensim

# Import utility files
from utils import read_df, remove_links, clean_sentence, save_object, load_object

#### Setup directories

If this is the first time doing this analysis, 
we first will set up all the directories we need
to save and load the models we will be using

In [2]:
import os
directories = ['objects', 'models', 'clusters', 'matricies']
for dirname in directories:
    if not os.path.exists(dirname):
        os.makedirs(dirname)

#### Name Model

Before begining the rest of our project, we select a name for our model.
This name will be used to save and load the files for this model

In [4]:
model_name = "PTSD_model"

#### Parse and Clean Data

We first parse and clean our data. Our data is assumed to be in csv format, 
in a directory labeled 'data'.

In [5]:
# Get the data from the csv
df = read_df('data')

In [6]:
# Do an inspection of our data to ensure nothing went wrong
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7057 entries, 0 to 208
Data columns (total 15 columns):
title           7057 non-null object
created_utc     7057 non-null float64
ups             7057 non-null float64
downs           7057 non-null float64
num_comments    7057 non-null float64
name            6076 non-null object
id              7057 non-null object
from            0 non-null object
from_id         0 non-null object
selftext        6215 non-null object
subreddit       7057 non-null object
score           7057 non-null float64
author          7057 non-null object
url             7057 non-null object
permalink       7057 non-null object
dtypes: float64(5), object(10)
memory usage: 882.1+ KB


In [7]:
df.head()

,title,created_utc,ups,downs,num_comments,name,id,from,from_id,selftext,subreddit,score,author,url,permalink
0,Posttraumatic stress disorder (Wikipedia Entry),1220412859,4,1,0,t3_6zcej,6zcej,NaN,NaN,NaN,ptsd,3,Crito,http://en.wikipedia.org/wiki/PTSD,/r/ptsd/comments/6zcej/posttraumatic_stress_di...
1,Psychiatric Service Dog Society,1220412757,4,1,0,t3_6zce5,6zce5,NaN,NaN,NaN,ptsd,3,Crito,http://www.psychdog.org/about_mission.html,/r/ptsd/comments/6zce5/psychiatric_service_dog...
2,PTSD leaves physical footprints on the brain,1220637551,4,1,0,t3_6zvkd,6zvkd,NaN,NaN,NaN,ptsd,3,Crito,http://www.sfgate.com/cgi-bin/article.cgi?f=/c...,/r/ptsd/comments/6zvkd/ptsd_leaves_physical_fo...
3,Computer therapy soothes symptoms in combat ve...,1220976763,5,0,0,t3_70hw3,70hw3,NaN,NaN,NaN,ptsd,5,Crito,http://www.signonsandiego.com/news/military/20...,/r/ptsd/comments/70hw3/computer_therapy_soothe...
4,"Trauma, PTSD Followed By Reduction In Region O...",1221046789,5,0,0,t3_70nr2,70nr2,NaN,NaN,NaN,ptsd,5,Crito,http://www.sciencedaily.com/releases/2008/08/0...,/r/ptsd/comments/70nr2/trauma_ptsd_followed_by...


In [8]:
# Clean the text in the dataframe
df = df.replace(np.nan, '', regex = True)
df = df.replace("\[deleted\]", '', regex = True)
df["rawtext"] = df["title"] + " " + df["selftext"]
df["cleantext"] = df["rawtext"].apply(remove_links).apply(clean_sentence)

In [9]:
# Check that the cleaning was successful
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7057 entries, 0 to 208
Data columns (total 17 columns):
title           7057 non-null object
created_utc     7057 non-null float64
ups             7057 non-null float64
downs           7057 non-null float64
num_comments    7057 non-null float64
name            7057 non-null object
id              7057 non-null object
from            7057 non-null object
from_id         7057 non-null object
selftext        7057 non-null object
subreddit       7057 non-null object
score           7057 non-null float64
author          7057 non-null object
url             7057 non-null object
permalink       7057 non-null object
rawtext         7057 non-null object
cleantext       7057 non-null object
dtypes: float64(5), object(12)
memory usage: 992.4+ KB


In [10]:
df.head()

,title,created_utc,ups,downs,num_comments,name,id,from,from_id,selftext,subreddit,score,author,url,permalink,rawtext,cleantext
0,Posttraumatic stress disorder (Wikipedia Entry),1220412859,4,1,0,t3_6zcej,6zcej,,,,ptsd,3,Crito,http://en.wikipedia.org/wiki/PTSD,/r/ptsd/comments/6zcej/posttraumatic_stress_di...,Posttraumatic stress disorder (Wikipedia Entry),posttraumatic stress disorder wikipedia entry
1,Psychiatric Service Dog Society,1220412757,4,1,0,t3_6zce5,6zce5,,,,ptsd,3,Crito,http://www.psychdog.org/about_mission.html,/r/ptsd/comments/6zce5/psychiatric_service_dog...,Psychiatric Service Dog Society,psychiatric service dog society
2,PTSD leaves physical footprints on the brain,1220637551,4,1,0,t3_6zvkd,6zvkd,,,,ptsd,3,Crito,http://www.sfgate.com/cgi-bin/article.cgi?f=/c...,/r/ptsd/comments/6zvkd/ptsd_leaves_physical_fo...,PTSD leaves physical footprints on the brain,ptsd leaves physical footprints on the brain
3,Computer therapy soothes symptoms in combat ve...,1220976763,5,0,0,t3_70hw3,70hw3,,,,ptsd,5,Crito,http://www.signonsandiego.com/news/military/20...,/r/ptsd/comments/70hw3/computer_therapy_soothe...,Computer therapy soothes symptoms in combat ve...,computer therapy soothes symptoms in combat ve...
4,"Trauma, PTSD Followed By Reduction In Region O...",1221046789,5,0,0,t3_70nr2,70nr2,,,,ptsd,5,Crito,http://www.sciencedaily.com/releases/2008/08/0...,/r/ptsd/comments/70nr2/trauma_ptsd_followed_by...,"Trauma, PTSD Followed By Reduction In Region O...",trauma ptsd followed by reduction in region o...


### Phrase Analysis

After parsing and cleaning the data we run the gensim phraser
tool on our text data to join phrases like "new york city" 
together to form the word "new_york_city"

In [11]:
# Get a stream of tokens
posts = df["cleantext"].apply(lambda str: str.split()).tolist()

In [12]:
# Train a phraseDetector to join two word phrases together
two_word_phrases = gensim.models.Phrases(posts)
two_word_phraser = gensim.models.phrases.Phraser(two_word_phrases)

In [13]:
# Train a phraseDetector to join three word phrases together
three_word_phrases = gensim.models.Phrases(two_word_phraser[posts])
three_word_phraser = gensim.models.phrases.Phraser(three_word_phrases)
posts = list(three_word_phraser[two_word_phraser[posts]])

In [14]:
# Update Data frame
df["phrasetext"] = df["cleantext"].apply(lambda str: " ".join(three_word_phraser[two_word_phraser[str.split()]]))

In [15]:
# Ensure posts contain same number of elements
len(posts) == len(df)

True

In [16]:
# Check that the dataframe was updated correctly
for i in range(len(posts)):
    if not " ".join(posts[i]) == list(df["phrasetext"])[i]:
        print("index :" + str(i) + " is incorrect")

### Data Saving

After cleaning and parsing all of our data, we can now
save it, so that we can analysis it later without having
to go through lengthy computations

In [17]:
save_object(posts, 'objects/', model_name + "-posts")
save_object(df, 'objects/', model_name + "-df")

### Initialize Word2Vec Model

After all of our data has been parsed and saved, 
we generate our Word2Vec Model

In [18]:
# Set the minimum word count to 10. This removes all words that appear less than 10 times in the data
minimum_word_count = 10
# Set skip gram to 1. This sets gensim to use the skip gram model instead of the Continuous Bag of Words model
skip_gram = 1
# Set Hidden layer size to 300.
hidden_layer_size = 300
# Set the window size to 5. 
window_size = 5
# Set hierarchical softmax to 1. This sets gensim to use hierarchical softmax
hierarchical_softmax = 1
# Set negative sampling to 20. This is good for relatively small data sets, but becomes harder for larger datasets
negative_sampling = 20

In [19]:
# Build the model
model = gensim.models.Word2Vec(posts, min_count = minimum_word_count, sg = skip_gram, size = hidden_layer_size,
                               window = window_size, hs = hierarchical_softmax, negative = negative_sampling)

### Basic Model test

After generating our model, we run some basic tests
to ensure that it has captured some semantic information results

In [20]:
model.most_similar(positive = ["kitten"])

[('vodka', 0.6811205744743347),
 ('daycare', 0.6777697205543518),
 ('japan', 0.6625845432281494),
 ('neighbours', 0.660164475440979),
 ('cancel', 0.6590190529823303),
 ('pair', 0.6582304239273071),
 ('hospice', 0.6498088836669922),
 ('owner', 0.646320104598999),
 ('sheets', 0.645109236240387),
 ('backyard', 0.6435533761978149)]

In [21]:
model.most_similar(positive = ["father", "woman"], negative = ["man"])

[('stepfather', 0.5219822525978088),
 ('mother', 0.5164777040481567),
 ('molested_by', 0.5076919794082642),
 ('ex_husband', 0.4981922507286072),
 ('schizophrenic', 0.4953431189060211),
 ('mentally_abusive', 0.49383193254470825),
 ('raised_by', 0.4865146279335022),
 ('half_brother', 0.48134884238243103),
 ('bio', 0.47938424348831177),
 ('physically_abusive', 0.47608622908592224)]

In [22]:
model.most_similar(positive = ["family", "obligation"], negative = ["love"])

[('police_officers', 0.47538650035858154),
 ('extended_family', 0.47335386276245117),
 ('relatives', 0.4723043441772461),
 ('aunts', 0.465665340423584),
 ('reputation', 0.46317481994628906),
 ('immediate_family', 0.46269842982292175),
 ('exs', 0.4467809498310089),
 ('cut_ties_with', 0.44380924105644226),
 ('norm', 0.4427856504917145),
 ('half_brother', 0.4408080577850342)]

### Save Model

After generating our model, and runing some basic tests,
we now save it so that we can analysis it later without having
to go through lengthy computations. We also delete and then reload
the model, as an example of how to do so.

In [23]:
model.save('models/' + model_name + '.model')
del model

In [24]:
model = gensim.models.Word2Vec.load('models/' + model_name + '.model')

### Generate Matricies

After generating our Word2Vec Model, we generate 
a collection of matricies that will be useful for
analysis. This includes a Words By feature matrix,
and a Post By Words Matrix. Note, we will use camelCase 
for matrix names, and only matrix names

In [25]:
# Initialize the list of words used
vocab_list = sorted(list(model.wv.vocab))

In [26]:
# Extract the word vectors
vecs = []
for word in vocab_list:
    vecs.append(model.wv[word].tolist())

In [27]:
# change array format into numpy array
WordsByFeatures = np.array(vecs)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer(vocabulary = vocab_list, analyzer = (lambda lst:list(map((lambda s:s), lst))), min_df = 0)

In [29]:
# Make Posts By Words Matrix
PostsByWords = countvec.fit_transform(posts)

### Basic Matrix tests

After generating our matricies, we run some basic tests
to ensure that they seem resaonable later without having
to go through lengthy computations

In [30]:
# Check that PostsByWords is the number of Posts by the number of words
PostsByWords.shape[0] == len(posts)

True

In [31]:
# check that the number of words is consistant for all matricies
PostsByWords.shape[1] == len(WordsByFeatures)

True

### Save Matricies

After generating our matricies, we save them so we can 
analyze them later without having to go through lengthy
computations.

In [32]:
save_object(PostsByWords,'matricies/', model_name + "-PostsByWords")
save_object(WordsByFeatures,'matricies/', model_name + "-WordsByFeatures")

### Generate Word Clusters

Now that we have generated and saved our matricies,
we will proceed to generate word clusters using 
kmeans clustering, and save them for later analysis.

In [33]:
from sklearn.cluster import KMeans
# get the fit for different values of K
test_points = [12] + list(range(25, 401, 25))
fit = []
for point in test_points:
    kmeans = KMeans(n_clusters = point, random_state = 42).fit(WordsByFeatures)
    save_object(kmeans, 'clusters/', model_name + "-words-cluster_model-" + str(point))
    fit.append(kmeans.inertia_)

In [34]:
save_object(fit, 'objects/', model_name + "-words" + "-fit")
save_object(test_points, 'objects/', model_name + "-words" + "-test_points")
del fit
del test_points